# Reference

https://github.com/khushwant18/Grasp-and-Lift-EEG-detection/blob/master/LSTM.ipynb

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense,BatchNormalization, Dropout, Input
from tensorflow.keras.layers import LSTM, Dense,BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
import glob
from scipy import signal
from scipy.fft import fft, fftfreq
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import utils

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %


# Without preprocessing

Training data

In [2]:
#read csv file from the path
xpath = 'grasp-and-lift-eeg-detection/subject1/train/data' 
xfiles = glob.glob(xpath + "/*.csv")
ypath = 'grasp-and-lift-eeg-detection/subject1/train/event' 
yfiles = glob.glob(ypath + "/*.csv")

xli = []
yli = []

for f in xfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    xli.append(df)

for f in yfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    yli.append(df)
    
#concatatenate all the dataframes in the li
x_all_df = pd.concat(xli, axis=0, ignore_index=True)
x_all_df = x_all_df.iloc[:,1:]
x_all_data=x_all_df.values

y_df = pd.concat(yli, axis=0, ignore_index=True)
y_df = y_df.iloc[:,1:]
y_data=y_df.values

In [3]:
x_all_data.shape

(1185498, 32)

Choose our target 11 channels:['T7','C3','Cz','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10']

In [4]:
target_channels = ['T7','C3','Cz','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10']
target_channels_index = []
all_channels = list(x_all_df.keys())
for item in target_channels:
    index = all_channels.index(item)
    target_channels_index.append(index)

print(target_channels_index)

[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [5]:
x_target_channels = []
for index in target_channels_index:
    x_target_channels.append(x_all_data[:,index])
    
x_target_channels = np.array(x_target_channels).T

In [6]:
x_data = x_target_channels

In [7]:
x_data.shape

(1185498, 11)

In [8]:
x_df = pd.DataFrame(x_data)
x_df

,0,1,2,3,4,5,6,7,8,9,10
0,-166,192,230,573,860,128,59,272,473,325,379
1,-138,201,233,554,846,185,47,269,455,307,368
2,-163,198,207,542,768,145,52,250,452,273,273
3,-310,212,221,542,808,115,41,276,432,258,241
4,-320,242,230,545,865,180,89,288,444,275,275
...,...,...,...,...,...,...,...,...,...,...,...
1185493,-351,164,-229,51,-148,-32,-384,-83,-66,-143,-128
1185494,-387,167,-230,37,-215,-85,-392,-92,-75,-192,-167
1185495,-309,162,-233,51,-274,-97,-421,-104,-85,-184,-180
1185496,-348,135,-229,70,-202,-51,-457,-103,-74,-152,-150


Testing data

In [9]:
#read csv file from the path
xtestpath = 'grasp-and-lift-eeg-detection/subject1/test/data' 
xtestfiles = glob.glob(xtestpath + "/*.csv")
ytestpath = 'grasp-and-lift-eeg-detection/subject1/test/event' 
ytestfiles = glob.glob(ytestpath + "/*.csv")

xtestli = []
ytestli = []

for f in xtestfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    xtestli.append(df)

for f in ytestfiles:
    df = pd.read_csv(f, index_col=None, header=0)
    ytestli.append(df)
    
#concatatenate all the dataframes in the li
x_ts_all_df = pd.concat(xtestli, axis=0, ignore_index=True)
x_ts_all_df = x_ts_all_df.iloc[:,1:]
x_ts_all_data=x_ts_all_df.values
# x_val_data=x_ts_df.values
# x_val_data=StandardScaler().fit_transform(x_val_data)

y_ts_df = pd.concat(ytestli, axis=0, ignore_index=True)
y_ts_df = y_ts_df.iloc[:,1:]
y_ts_data=y_ts_df.values

In [10]:
x_ts_all_data.shape

(236894, 32)

Choose our target 11 channels:['T7','C3','Cz','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10']

In [11]:
x_ts_target_channels = []
for index in target_channels_index:
    x_ts_target_channels.append(x_ts_all_data[:,index])
    
x_ts_target_channels = np.array(x_ts_target_channels).T

In [12]:
x_ts_data = x_ts_target_channels

In [13]:
x_ts_data.shape

(236894, 11)

In [14]:
x_ts_df = pd.DataFrame(x_ts_data)
x_ts_df

,0,1,2,3,4,5,6,7,8,9,10
0,67,270,-215,89,-37,-153,34,-39,114,15,-25
1,81,272,-220,102,29,-118,37,-44,124,22,36
2,76,285,-195,113,76,13,80,-20,134,56,152
3,69,249,-202,89,160,-159,5,-39,106,37,75
4,125,257,-221,121,200,-62,39,-38,125,62,69
...,...,...,...,...,...,...,...,...,...,...,...
236889,13,322,-45,226,238,74,-11,-30,-3,135,298
236890,37,320,-23,229,231,32,-13,1,-6,128,287
236891,79,315,-40,218,182,29,-41,-18,-31,91,252
236892,80,302,-42,204,176,61,-55,-24,-28,88,294


Filter [0 0 0 0 0 0] from the data

In [15]:
time_steps=1000
subsample=50

In [16]:
def filter_dataset(x_dataset, y_dataset):
    x_data_filter = []
    y_data_filter = []
    for i in range(x_dataset.shape[0]-time_steps):
        if 1 in y_dataset[i+time_steps]:
            x_data_filter.append(x_dataset[i:i+time_steps:subsample])
            y_data_filter.append(y_dataset[i+time_steps])
    x_data_filter = np.array(x_data_filter)
    y_data_filter = np.array(y_data_filter)
    return x_data_filter, y_data_filter

In [17]:
x_data_filter, y_data_filter = filter_dataset(x_data, y_data)

In [18]:
x_data_filter.shape

(132965, 20, 11)

In [19]:
y_data_filter.shape

(132965, 6)

In [20]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_ts_data, y_ts_data)

In [21]:
x_ts_data_filter.shape

(45838, 20, 11)

In [22]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [23]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [24]:
x_data_filter.shape

(132965, 20, 11)

In [25]:
y_data_filter.shape

(132965, 6)

Model

In [26]:
def LSTM_model(featr):
    model = Sequential()
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128, return_sequences = True, input_shape = (time_steps//subsample, featr)))
    model.add(Dropout(0.5))
    model.add(LSTM(128))
    model.add(Dense(32, activation = "relu"))
    model.add(Dense(6, activation = "sigmoid"))
    
    optimizer = Adam(lr = 0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [28]:
model_without_preprocessing = LSTM_model(11)
model_without_preprocessing.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [29]:
model_without_preprocessing.save("modelop/LSTM/LSTM_without_preprocessing.h5")

Evaluate

In [30]:
model = load_model("modelop/LSTM/LSTM_without_preprocessing.h5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [31]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 14s 9ms/step - loss: 1.5203 - accuracy: 0.3589


[1.5203118324279785, 0.3589380085468292]

# With lowpass filter

Training data with filter

In [55]:
x_data_df=x_df.copy()

In [56]:
fs = 500  # Sampling frequency
fc = 35  # Cut-off frequency of the filter
w = fc / (fs / 2) # Normalize the frequency
for col in x_data_df.columns:
    b, a = signal.butter(8, w, 'low')
    x_data_df[col] = signal.filtfilt(b, a, x_data_df[col].values)

In [57]:
x_data_df.values

array([[-166.82819025,  192.51980174,  230.36361371, ...,  473.48230623,
         326.06142442,  379.98444149],
       [-174.27948695,  200.77913259,  225.6359968 , ...,  465.44873684,
         321.13388838,  367.72668921],
       [-181.06571617,  208.07644964,  220.75248357, ...,  457.52450939,
         315.48595846,  354.82668766],
       ...,
       [-354.29637221,  165.51670265, -229.37401964, ...,  -87.2687435 ,
        -177.82331846, -162.28588387],
       [-377.90769853,  154.77682066, -228.63186945, ...,  -92.2583437 ,
        -182.62525868, -169.08319706],
       [-402.48216844,  144.31783962, -227.76996076, ...,  -97.85267257,
        -187.13373086, -175.5942452 ]])

In [58]:
x_data_df.values.shape

(1185498, 11)

Testing data with filter

In [59]:
x_test_df=x_ts_df.copy()

In [60]:
for col in x_test_df.columns:
    b, a = signal.butter(8, w, 'low')
    x_test_df[col] = signal.filtfilt(b, a, x_test_df[col].values)

In [61]:
x_test_df.values

array([[  67.34267675,  269.65820415, -215.17929161, ...,  114.10565348,
          14.28476537,  -25.1310076 ],
       [  78.59154509,  266.38083786, -212.51578362, ...,  115.51932772,
          24.89164716,   13.46311616],
       [  88.78362445,  262.88488934, -210.11379572, ...,  116.59767179,
          34.84600294,   48.26187557],
       ...,
       [  89.56179842,  310.27267273,  -44.44579712, ...,  -37.19078695,
         105.28696398,  283.62657741],
       [  75.77247933,  299.05110992,  -48.21155977, ...,  -49.07019079,
          97.92210305,  270.45770314],
       [  60.49225403,  287.30892857,  -51.93974335, ...,  -61.86529521,
          89.00335433,  255.38677467]])

In [62]:
x_test_df.values.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [40]:
x_data_filter, y_data_filter = filter_dataset(x_data_df.values, y_data)

In [41]:
x_data_filter.shape

(132965, 20, 11)

In [42]:
y_data_filter.shape

(132965, 6)

In [43]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_df.values, y_ts_data)

In [44]:
x_ts_data_filter.shape

(45838, 20, 11)

In [45]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [46]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [47]:
x_data_filter.shape

(132965, 20, 11)

In [48]:
y_data_filter.shape

(132965, 6)

Model

In [49]:
model_with_filter = LSTM_model(11)
model_with_filter.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [50]:
model_with_filter.save("modelop/LSTM/LSTM_with_filter.h5")

Evaluate

In [51]:
model = load_model("modelop/LSTM/LSTM_with_filter.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [52]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 14s 9ms/step - loss: 1.3859 - accuracy: 0.3900


[1.3858853578567505, 0.38996028900146484]

# With lowpass filter and standardization

Training data with filter and standardization

In [53]:
Xsc = StandardScaler().fit_transform(x_data_df.values)
Xsc=pd.DataFrame(Xsc)
y=pd.DataFrame(y_data)

In [54]:
Xsc.values

array([[-0.92215592, -0.72194023,  2.71379359, ...,  2.30970005,
         1.71593394,  0.56826548],
       [-0.94840043, -0.65151327,  2.66568898, ...,  2.25704488,
         1.68692048,  0.53007825],
       [-0.97230248, -0.58928937,  2.61599808, ...,  2.20510639,
         1.65366532,  0.48989018],
       ...,
       [-1.5824451 , -0.9521945 , -1.96414498, ..., -1.3656823 ,
        -1.25095259, -1.1210983 ],
       [-1.6656075 , -1.04377301, -1.95659342, ..., -1.39838609,
        -1.27922654, -1.14227434],
       [-1.75216222, -1.13295629, -1.9478233 , ..., -1.43505352,
        -1.30577254, -1.16255856]])

In [55]:
Xsc.values.shape

(1185498, 11)

Testing data with filter and standardization

In [56]:
x_test_sc=StandardScaler().fit_transform(x_test_df.values)
x_test_sc=pd.DataFrame(x_test_sc)
y_test=pd.DataFrame(y_ts_data)

In [57]:
x_test_sc.values

array([[-0.30725723, -0.61228674, -1.9309016 , ...,  0.77142367,
         0.19211797, -0.68288217],
       [-0.23340494, -0.64543689, -1.90234217, ...,  0.78691927,
         0.27681223, -0.51723425],
       [-0.1664908 , -0.68079798, -1.87658689, ...,  0.79873924,
         0.35629617, -0.36787623],
       ...,
       [-0.16138185, -0.20147644, -0.10021382, ..., -0.88697048,
         0.91875602,  0.64232089],
       [-0.25191298, -0.31498115, -0.14059217, ..., -1.01718328,
         0.85994878,  0.58579942],
       [-0.35223236, -0.43375185, -0.18056757, ..., -1.15743328,
         0.78873399,  0.52111424]])

In [58]:
x_test_sc.values.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [59]:
x_data_filter, y_data_filter = filter_dataset(Xsc.values, y_data)

In [60]:
x_data_filter.shape

(132965, 20, 11)

In [61]:
y_data_filter.shape

(132965, 6)

In [62]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_sc.values, y_ts_data)

In [63]:
x_ts_data_filter.shape

(45838, 20, 11)

In [64]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [65]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [66]:
x_data_filter.shape

(132965, 20, 11)

In [67]:
y_data_filter.shape

(132965, 6)

Model

In [68]:
model_with_filter_standard = LSTM_model(11)
model_with_filter_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [69]:
model_with_filter_standard.save("modelop/LSTM/LSTM_with_filter_standard.h5")

Evaluate

In [70]:
model = load_model("modelop/LSTM/LSTM_with_filter_standard.h5")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [71]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 13s 9ms/step - loss: 1.6988 - accuracy: 0.4310


[1.6987732648849487, 0.4310397505760193]

# With lowpass filter 、linear regression and standardization (segment length = 50)

Training data with filter, Linear regression and standardization

In [27]:
def linearregression_seg(signal):
    model = LinearRegression(fit_intercept=True)
    time = np.arange(0, len(signal), 1)
    model.fit(time[:, np.newaxis], signal)
    xfit = np.linspace(time[0], time[len(time)-1], len(signal))
    yfit = model.predict(xfit[:, np.newaxis])
    
    return yfit

def linearregression_allsignal(signal, seg_length):
    new_signal = []
    for i in range((len(signal)//seg_length)+1):
        if (i+1)*seg_length<len(signal):
            new_signal += linearregression_seg(signal[i*seg_length:(i+1)*seg_length]).tolist()
        else:
            new_signal += linearregression_seg(signal[i*seg_length:len(signal)]).tolist()
    
    return np.asarray(new_signal)

In [73]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 50)

In [74]:
x_train_filter_LR

array([[-174.26169504,  215.27043477,  208.61237137, ...,  437.85560613,
         249.5223138 ,  279.04812703],
       [-174.38144154,  215.04221594,  207.25176311, ...,  436.96043371,
         249.02018265,  277.75359057],
       [-174.50118804,  214.81399712,  205.89115485, ...,  436.06526129,
         248.51805149,  276.45905411],
       ...,
       [-315.63144689,  234.17528534, -247.14962021, ...,  -74.5968628 ,
        -149.64146428, -144.19563708],
       [-319.65550708,  234.3081823 , -249.03301887, ...,  -75.17422211,
        -151.55910395, -146.94694633],
       [-323.67956727,  234.44107926, -250.91641752, ...,  -75.75158142,
        -153.47674362, -149.69825559]])

In [75]:
x_train_filter_LR.shape

(1185498, 11)

In [76]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [77]:
x_train_filter_LR_st.values

array([[-0.96133839, -0.53839238,  2.51757981, ...,  2.105295  ,
         1.29395529,  0.25646106],
       [-0.96176593, -0.5403769 ,  2.50359582, ...,  2.09934544,
         1.29093169,  0.25238604],
       [-0.96219348, -0.54236141,  2.48961184, ...,  2.09339587,
         1.2879081 ,  0.24831103],
       ...,
       [-1.46608841, -0.37400206, -2.16662572, ..., -1.3006071 ,
        -1.10961611, -1.0758486 ],
       [-1.48045602, -0.37284643, -2.18598281, ..., -1.30444439,
        -1.12116321, -1.08450932],
       [-1.49482362, -0.3716908 , -2.2053399 , ..., -1.30828168,
        -1.13271031, -1.09317004]])

In [78]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with filter, Linear regression and standardization

In [79]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 50)

In [80]:
x_test_filter_LR

array([[  55.79972655,  234.50475845, -219.80506913, ...,   98.4959142 ,
          25.02874466,   44.13973947],
       [  56.75109696,  234.63054497, -219.85635771, ...,  100.2700706 ,
          28.14686971,   47.11614045],
       [  57.70246737,  234.75633149, -219.90764629, ...,  102.04422701,
          31.26499476,   50.09254144],
       ...,
       [ 101.99392811,  345.02764695,  -27.01159925, ...,  -25.92839233,
          89.30772653,  263.88332668],
       [ 103.79377216,  345.22757456,  -28.01039187, ...,  -26.50458388,
          90.64330536,  266.2304748 ],
       [ 105.59361621,  345.42750216,  -29.0091845 , ...,  -27.08077543,
          91.97888419,  268.57762292]])

In [81]:
x_test_filter_LR.shape

(236894, 11)

In [82]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [83]:
x_test_filter_LR_st.values

array([[-0.40073606, -0.99420311, -2.00016447, ...,  0.62259952,
         0.28859987, -0.39251709],
       [-0.39420147, -0.99289617, -2.00071987, ...,  0.64276812,
         0.31445558, -0.37951201],
       [-0.38766687, -0.99158922, -2.00127527, ...,  0.66293671,
         0.34031129, -0.36650693],
       ...,
       [-0.08344591,  0.15415037,  0.08758517, ..., -0.79185481,
         0.82160574,  0.56763021],
       [-0.07108348,  0.15622765,  0.0767693 , ..., -0.79840495,
         0.83268045,  0.57788584],
       [-0.05872105,  0.15830494,  0.06595343, ..., -0.80495509,
         0.84375516,  0.58814146]])

In [84]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [85]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [86]:
x_data_filter.shape

(132965, 20, 11)

In [87]:
y_data_filter.shape

(132965, 6)

In [88]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [89]:
x_ts_data_filter.shape

(45838, 20, 11)

In [90]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [91]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [92]:
x_data_filter.shape

(132965, 20, 11)

In [93]:
y_data_filter.shape

(132965, 6)

Model

In [94]:
model_with_filter_LG_standard = LSTM_model(11)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [95]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR50_standard.h5")

Evaluate

In [96]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR50_standard.h5")
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [97]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 8ms/step - loss: 1.7347 - accuracy: 0.4690


[1.7346899509429932, 0.4689558744430542]

# With lowpass filter 、linear regression and standardization (segment length = 100)

Training data with filter, Linear regression and standardization

In [98]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 100)

In [99]:
x_train_filter_LR

array([[-188.31563189,  198.52856702,  186.7549258 , ...,  429.22093657,
         241.51255544,  263.67050519],
       [-188.05050895,  198.7971032 ,  186.42028041, ...,  428.75346893,
         241.19964664,  263.00912785],
       [-187.78538602,  199.06563938,  186.08563503, ...,  428.2860013 ,
         240.88673785,  262.34775051],
       ...,
       [-274.45948826,  209.56165424, -243.37021402, ...,  -72.71941712,
        -126.56051834, -111.85208223],
       [-276.19298292,  208.66612723, -245.54662226, ...,  -73.13039832,
        -127.12166467, -112.50007615],
       [-277.92647758,  207.77060021, -247.72303049, ...,  -73.54137951,
        -127.682811  , -113.14807007]])

In [100]:
x_train_filter_LR.shape

(1185498, 11)

In [101]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [102]:
x_train_filter_LR_st.values

array([[-1.01824116, -0.69136649,  2.31261873, ...,  2.06875722,
         1.26444687,  0.20967836],
       [-1.01728827, -0.68900615,  2.30914981, ...,  2.06561867,
         1.26253437,  0.20758019],
       [-1.01633537, -0.68664581,  2.30568088, ...,  2.06248011,
         1.26062187,  0.20548202],
       ...,
       [-1.32785593, -0.5943895 , -2.14604833, ..., -1.30124408,
        -0.98522185, -0.98163791],
       [-1.33408639, -0.60226087, -2.16860893, ..., -1.30400339,
        -0.98865158, -0.98369362],
       [-1.34031685, -0.61013224, -2.19116953, ..., -1.30676269,
        -0.99208132, -0.98574934]])

In [103]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with filter, Linear regression and standardization

In [104]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 100)

In [105]:
x_test_filter_LR

array([[  91.25055841,  242.63802923, -225.82540226, ...,  134.69518214,
          95.53546924,  116.63384089],
       [  90.59443823,  242.53798665, -225.68413539, ...,  134.68162001,
          95.26789232,  116.17283597],
       [  89.93831805,  242.43794408, -225.54286851, ...,  134.66805789,
          95.00031541,  115.71183105],
       ...,
       [  74.36783913,  341.92477945,  -13.73262517, ...,  -12.89820709,
          71.43136741,  219.77734593],
       [  74.88840944,  342.10043768,  -13.90136348, ...,  -12.83657942,
          71.71127048,  219.72800511],
       [  75.40897975,  342.2760959 ,  -14.07010178, ...,  -12.77495175,
          71.99117355,  219.67866429]])

In [106]:
x_test_filter_LR.shape

(236894, 11)

In [107]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [108]:
x_test_filter_LR_st.values

array([[-0.16128384, -0.9239655 , -2.08167486, ...,  1.0613791 ,
         0.89622237, -0.07677653],
       [-0.16590644, -0.92502126, -2.080133  , ...,  1.06122086,
         0.89394523, -0.07881783],
       [-0.17052903, -0.92607703, -2.07859114, ...,  1.06106262,
         0.89166808, -0.08085913],
       ...,
       [-0.2802285 ,  0.12382317,  0.23321038, ..., -0.66070108,
         0.69109065,  0.37993672],
       [-0.2765609 ,  0.12567692,  0.23136869, ..., -0.65998202,
         0.69347269,  0.37971824],
       [-0.2728933 ,  0.12753066,  0.22952699, ..., -0.65926297,
         0.69585474,  0.37949977]])

In [109]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [110]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [111]:
x_data_filter.shape

(132965, 20, 11)

In [112]:
y_data_filter.shape

(132965, 6)

In [113]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [114]:
x_ts_data_filter.shape

(45838, 20, 11)

In [115]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [116]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [117]:
x_data_filter.shape

(132965, 20, 11)

In [118]:
y_data_filter.shape

(132965, 6)

Model

In [119]:
model_with_filter_LG_standard = LSTM_model(11)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [120]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR100_standard.h5")

Evaluate

In [121]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR100_standard.h5")
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [122]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 14s 9ms/step - loss: 1.5767 - accuracy: 0.4789


[1.5766746997833252, 0.47892579436302185]

# With lowpass filter 、linear regression and standardization (segment length = 25)

Training data with filter, Linear regression and standardization

In [123]:
x_train_filter_LR = linearregression_allsignal(x_data_df.values, 25)

In [124]:
x_train_filter_LR

array([[-193.86041918,  202.19067761,  219.60198596, ...,  457.04038024,
         312.86905409,  339.67607895],
       [-192.6283537 ,  202.96425892,  217.21591506, ...,  454.16349918,
         306.27167493,  332.62619476],
       [-191.39628822,  203.73784022,  214.82984417, ...,  451.28661813,
         299.67429576,  325.57631056],
       ...,
       [-327.29509528,  193.62120008, -230.35346638, ...,  -90.86146283,
        -158.19623261, -144.93869853],
       [-333.56886036,  187.96626555, -231.46545684, ...,  -93.91025203,
        -162.24272074, -147.8897084 ],
       [-339.84262544,  182.31133102, -232.5774473 , ...,  -96.95904123,
        -166.28920887, -150.84071827]])

In [125]:
x_train_filter_LR.shape

(1185498, 11)

In [126]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [127]:
x_train_filter_LR_st.values

array([[-1.02561802, -0.64613525,  2.61642159, ...,  2.21566011,
         1.65521987,  0.44492722],
       [-1.02124332, -0.63947028,  2.59202967, ...,  2.19668635,
         1.61597211,  0.42285338],
       [-1.01686862, -0.6328053 ,  2.56763775, ...,  2.1777126 ,
         1.57672435,  0.40077954],
       ...,
       [-1.49940511, -0.71996763, -1.98330595, ..., -1.39789088,
        -1.1471439 , -1.07244663],
       [-1.5216814 , -0.76868907, -1.99467342, ..., -1.41799842,
        -1.17121643, -1.08168652],
       [-1.54395768, -0.8174105 , -2.00604089, ..., -1.43810595,
        -1.19528895, -1.0909264 ]])

In [128]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with filter, Linear regression and standardization

In [129]:
x_test_filter_LR = linearregression_allsignal(x_test_df.values, 25)

In [130]:
x_test_filter_LR

array([[  87.17712512,  251.72521793, -208.93502212, ...,  107.47006315,
          21.68458679,   26.33306532],
       [  85.31335753,  250.57686109, -209.89488147, ...,  108.3517209 ,
          25.11603257,   31.60198384],
       [  83.44958993,  249.42850426, -210.85474082, ...,  109.23337865,
          28.54747836,   36.87090237],
       ...,
       [  85.32765873,  316.48669496,  -40.48320542, ...,  -28.78687805,
         102.74472893,  279.6035681 ],
       [  85.65246175,  313.08447658,  -43.31860354, ...,  -30.78666366,
         104.91771381,  284.6576509 ],
       [  85.97726478,  309.68225819,  -46.15400165, ...,  -32.78644927,
         107.09069869,  289.7117337 ]])

In [131]:
x_test_filter_LR.shape

(236894, 11)

In [132]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [133]:
x_test_filter_LR_st.values

array([[-0.18169856, -0.8049283 , -1.87292641, ...,  0.71040256,
         0.25555138, -0.46609141],
       [-0.19425689, -0.81670845, -1.88326807, ...,  0.72022862,
         0.28342506, -0.44327647],
       [-0.20681522, -0.82848861, -1.89360972, ...,  0.73005468,
         0.31129875, -0.42046153],
       ...,
       [-0.19416053, -0.14058757, -0.05800305, ..., -0.80817857,
         0.91400419,  0.63059508],
       [-0.19197196, -0.17548844, -0.08855203, ..., -0.83046614,
         0.93165538,  0.65247976],
       [-0.18978339, -0.21038932, -0.119101  , ..., -0.85275371,
         0.94930657,  0.67436444]])

In [134]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [135]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [136]:
x_data_filter.shape

(132965, 20, 11)

In [137]:
y_data_filter.shape

(132965, 6)

In [138]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [139]:
x_ts_data_filter.shape

(45838, 20, 11)

In [140]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [141]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [142]:
x_data_filter.shape

(132965, 20, 11)

In [143]:
y_data_filter.shape

(132965, 6)

Model

In [144]:
model_with_filter_LG_standard = LSTM_model(11)
model_with_filter_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [145]:
model_with_filter_LG_standard.save("modelop/LSTM/LSTM_with_filter_LR25_standard.h5")

Evaluate

In [146]:
model = load_model("modelop/LSTM/LSTM_with_filter_LR25_standard.h5")
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [147]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 14s 9ms/step - loss: 1.3832 - accuracy: 0.4807


[1.3831630945205688, 0.4806710481643677]

# With linear regression and standardization (segment length = 25)

Training data with Linear regression and standardization

In [148]:
x_train_filter_LR = linearregression_allsignal(x_data, 25)

In [149]:
x_train_filter_LR

array([[-190.36      ,  199.33538462,  219.60615385, ...,  453.62153846,
         298.28615385,  326.82153846],
       [-190.        ,  200.46076923,  217.30230769, ...,  451.02307692,
         292.86230769,  320.49307692],
       [-189.64      ,  201.58615385,  214.99846154, ...,  448.42461538,
         287.43846154,  314.16461538],
       ...,
       [-326.54644269,  189.16106719, -232.01284585, ...,  -89.19466403,
        -157.97332016, -150.86857708],
       [-332.93083004,  183.22727273, -233.38142292, ...,  -92.34189723,
        -162.14426877, -154.8201581 ],
       [-339.31521739,  177.29347826, -234.75      , ...,  -95.48913043,
        -166.31521739, -158.77173913]])

In [150]:
x_train_filter_LR.shape

(1185498, 11)

In [151]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [152]:
x_train_filter_LR_st.values

array([[-1.0123548 , -0.67048552,  2.61601355, ...,  2.19277581,
         1.56758053,  0.40453946],
       [-1.01107759, -0.66079311,  2.59246624, ...,  2.17564092,
         1.53533233,  0.38473128],
       [-1.00980039, -0.6511007 ,  2.56891893, ...,  2.15850604,
         1.50308414,  0.3649231 ],
       ...,
       [-1.49551447, -0.75811211, -1.99992473, ..., -1.38668544,
        -1.1451702 , -1.0906379 ],
       [-1.51816488, -0.80921708, -2.01391278, ..., -1.40743905,
        -1.16996913, -1.10300641],
       [-1.54081529, -0.86032205, -2.02790083, ..., -1.42819267,
        -1.19476805, -1.11537492]])

In [153]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with Linear regression and standardization

In [154]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 25)

In [155]:
x_test_filter_LR

array([[  87.78153846,  255.60615385, -207.40615385, ...,  111.90153846,
          24.72923077,   47.77846154],
       [  85.70307692,  254.13230769, -208.56230769, ...,  112.38307692,
          27.86846154,   51.20692308],
       [  83.62461538,  252.65846154, -209.71846154, ...,  112.86461538,
          31.00769231,   54.63538462],
       ...,
       [  73.18070175,  316.60350877,  -39.08596491, ...,  -24.58070175,
         101.59298246,  275.82982456],
       [  71.37192982,  313.18596491,  -41.72982456, ...,  -26.34035088,
         103.40701754,  279.72280702],
       [  69.56315789,  309.76842105,  -44.37368421, ...,  -28.1       ,
         105.22105263,  283.61578947]])

In [156]:
x_test_filter_LR.shape

(236894, 11)

In [157]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [158]:
x_test_filter_LR_st.values

array([[-0.17729599, -0.76481738, -1.85610736, ...,  0.75949226,
         0.28014457, -0.37301442],
       [-0.19127505, -0.77993057, -1.86856159, ...,  0.7648569 ,
         0.30563204, -0.35817751],
       [-0.20525412, -0.79504375, -1.88101583, ...,  0.77022154,
         0.3311195 , -0.34334061],
       ...,
       [-0.27549651, -0.1393354 , -0.04294129, ..., -0.76100588,
         0.9042027 ,  0.61389367],
       [-0.28766173, -0.17437974, -0.07142127, ..., -0.78060948,
         0.91893088,  0.63074083],
       [-0.29982694, -0.20942408, -0.09990126, ..., -0.80021308,
         0.93365907,  0.64758798]])

In [159]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [160]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [161]:
x_data_filter.shape

(132965, 20, 11)

In [162]:
y_data_filter.shape

(132965, 6)

In [163]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [164]:
x_ts_data_filter.shape

(45838, 20, 11)

In [165]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [166]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [167]:
x_data_filter.shape

(132965, 20, 11)

In [168]:
y_data_filter.shape

(132965, 6)

Model

In [169]:
model_with_LG_standard = LSTM_model(11)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_21 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_22 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_23 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 128)              

In [170]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR25_standard.h5")

Evaluate

In [171]:
model = load_model("modelop/LSTM/LSTM_with_LR25_standard.h5")
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_21 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_22 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_30 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_23 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_31 (LSTM)               (None, 128)              

In [172]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 13s 9ms/step - loss: 1.5244 - accuracy: 0.4758


[1.5244489908218384, 0.4758497178554535]

# With linear regression and standardization (segment length = 50)

Training data with Linear regression and standardization

In [173]:
x_train_filter_LR = linearregression_allsignal(x_data, 50)

In [174]:
x_train_filter_LR

array([[-173.35764706,  216.29882353,  209.52941176, ...,  437.19058824,
         242.23058824,  270.54352941],
       [-173.65080432,  216.0082593 ,  208.13392557, ...,  436.29954382,
         242.01138055,  269.54583433],
       [-173.94396158,  215.71769508,  206.73843938, ...,  435.4084994 ,
         241.79217287,  268.54813926],
       ...,
       [-315.30997612,  231.89472065, -247.01461861, ...,  -71.82944348,
        -148.36191924, -143.97812636],
       [-319.36799826,  231.96734332, -248.88060863, ...,  -72.28759589,
        -150.22177594, -146.68931828],
       [-323.42602041,  232.03996599, -250.74659864, ...,  -72.7457483 ,
        -152.08163265, -149.4005102 ]])

In [175]:
x_train_filter_LR.shape

(1185498, 11)

In [176]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [177]:
x_train_filter_LR_st.values

array([[-0.95791647, -0.5294007 ,  2.52691159, ...,  2.10078859,
         1.24983654,  0.22967138],
       [-0.95896295, -0.53192712,  2.51256967, ...,  2.09486671,
         1.2485168 ,  0.22653104],
       [-0.96000943, -0.53445354,  2.49822774, ...,  2.08894482,
         1.24719706,  0.22339069],
       ...,
       [-1.46464392, -0.39379646, -2.16515832, ..., -1.28216136,
        -1.10172432, -1.07507608],
       [-1.47912985, -0.39316502, -2.18433578, ..., -1.28520625,
        -1.11292159, -1.08360982],
       [-1.49361577, -0.39253358, -2.20351324, ..., -1.28825113,
        -1.12411885, -1.09214357]])

In [178]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with Linear regression and standardization

In [179]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 50)

In [180]:
x_test_filter_LR

array([[  55.34823529,  236.25647059, -219.95294118, ...,   99.72      ,
          26.75529412,   50.63058824],
       [  56.29647059,  236.3104922 , -219.99812725, ...,  101.45469388,
          29.78569028,   53.41627851],
       [  57.24470588,  236.36451381, -220.04331333, ...,  103.18938776,
          32.81608643,   56.20196879],
       ...,
       [ 101.7615222 ,  345.65221987,  -26.7832981 , ...,  -23.82980973,
          88.99196617,  263.86215645],
       [ 103.65954898,  345.89429175,  -27.7931642 , ...,  -24.36187456,
          90.27477097,  266.16744186],
       [ 105.55757576,  346.13636364,  -28.8030303 , ...,  -24.89393939,
          91.55757576,  268.47272727]])

In [181]:
x_test_filter_LR.shape

(236894, 11)

In [182]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [183]:
x_test_filter_LR_st.values

array([[-0.40370491, -0.97595244, -2.00170815, ...,  0.63647018,
         0.30290783, -0.36410687],
       [-0.39719396, -0.97539118, -2.00219745, ...,  0.65618885,
         0.32803541, -0.35193678],
       [-0.390683  , -0.97482991, -2.00268675, ...,  0.67590752,
         0.35316299, -0.33976669],
       ...,
       [-0.08501328,  0.16063062,  0.09005449, ..., -0.76794943,
         0.81896481,  0.56745652],
       [-0.07198069,  0.16314566,  0.07911903, ..., -0.77399753,
         0.82960164,  0.57752782],
       [-0.0589481 ,  0.1656607 ,  0.06818356, ..., -0.78004564,
         0.84023846,  0.58759912]])

In [184]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [185]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [186]:
x_data_filter.shape

(132965, 20, 11)

In [187]:
y_data_filter.shape

(132965, 6)

In [188]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [189]:
x_ts_data_filter.shape

(45838, 20, 11)

In [190]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [191]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [192]:
x_data_filter.shape

(132965, 20, 11)

In [193]:
y_data_filter.shape

(132965, 6)

Model

In [194]:
model_with_LG_standard = LSTM_model(11)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_24 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_25 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_26 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)              

In [195]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR50_standard.h5")

Evaluate

In [196]:
model = load_model("modelop/LSTM/LSTM_with_LR50_standard.h5")
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_24 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_33 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_25 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_26 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)              

In [197]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 14s 9ms/step - loss: 1.4945 - accuracy: 0.5181


[1.494529366493225, 0.5181072354316711]

# With linear regression and standardization (segment length = 100)

Training data with Linear regression and standardization

In [28]:
x_train_filter_LR = linearregression_allsignal(x_data, 100)

In [29]:
x_train_filter_LR

array([[-188.5229703 ,  198.40811881,  186.82851485, ...,  428.25267327,
         239.16079208,  260.28475248],
       [-188.27927393,  198.67987399,  186.4970297 , ...,  427.80534653,
         238.87794779,  259.67192919],
       [-188.03557756,  198.95162916,  186.16554455, ...,  427.3580198 ,
         238.59510351,  259.05910591],
       ...,
       [-273.28774171,  208.55591684, -243.63119943, ...,  -71.28556127,
        -125.75016736, -111.48866744],
       [-274.99482942,  207.64994358, -245.81374443, ...,  -71.66782702,
        -126.29252338, -112.11910181],
       [-276.70191713,  206.74397032, -247.99628942, ...,  -72.05009276,
        -126.83487941, -112.74953618]])

In [30]:
x_train_filter_LR.shape

(1185498, 11)

In [31]:
x_train_filter_LR_st = StandardScaler().fit_transform(x_train_filter_LR)
x_train_filter_LR_st = pd.DataFrame(x_train_filter_LR_st)
y=pd.DataFrame(y_data)

In [32]:
x_train_filter_LR_st.values

array([[-1.01892734, -0.69241475,  2.31338299, ...,  2.06223554,
         1.25000457,  0.19893308],
       [-1.01805151, -0.69002615,  2.30994682, ...,  2.05923224,
         1.24827592,  0.19698899],
       [-1.01717567, -0.68763755,  2.30651065, ...,  2.05622894,
         1.24654726,  0.1950449 ],
       ...,
       [-1.32356783, -0.60322045, -2.14875512, ..., -1.29160432,
        -0.98021489, -0.98046252],
       [-1.32970302, -0.61118352, -2.17137934, ..., -1.29417081,
        -0.9835296 , -0.98246248],
       [-1.33583822, -0.61914659, -2.19400357, ..., -1.2967373 ,
        -0.9868443 , -0.98446244]])

In [33]:
x_train_filter_LR_st.shape

(1185498, 11)

Testing data with Linear regression and standardization

In [34]:
x_test_filter_LR = linearregression_allsignal(x_ts_data, 100)

In [35]:
x_test_filter_LR

array([[  90.58653465,  243.36019802, -225.88316832, ...,  134.99168317,
          95.65306931,  120.79584158],
       [  89.94155416,  243.24706271, -225.74088209, ...,  134.97791179,
          95.38735074,  120.26138014],
       [  89.29657366,  243.13392739, -225.59859586, ...,  134.96414041,
          95.12163216,  119.72691869],
       ...,
       [  73.1834122 ,  342.38908355,  -13.72867103, ...,  -11.61580031,
          71.71336199,  219.15663042],
       [  73.68621898,  342.57808041,  -13.90330528, ...,  -11.53544775,
          71.99833833,  219.07719539],
       [  74.18902576,  342.76707727,  -14.07793953, ...,  -11.45509518,
          72.28331467,  218.99776036]])

In [36]:
x_test_filter_LR.shape

(236894, 11)

In [37]:
x_test_filter_LR_st = StandardScaler().fit_transform(x_test_filter_LR)
x_test_filter_LR_st = pd.DataFrame(x_test_filter_LR_st)
y_test=pd.DataFrame(y_ts_data)

In [38]:
x_test_filter_LR_st.values

array([[-0.16594993, -0.91630073, -2.0822818 , ...,  1.0647969 ,
         0.89723834, -0.05834776],
       [-0.17049375, -0.91749461, -2.08072884, ...,  1.06463623,
         0.89497697, -0.06071426],
       [-0.17503757, -0.91868848, -2.07917587, ...,  1.06447555,
         0.8927156 , -0.06308076],
       ...,
       [-0.28855305,  0.12871598,  0.23325053, ..., -0.64571696,
         0.69350208,  0.37717656],
       [-0.28501083,  0.1307104 ,  0.23134451, ..., -0.64477946,
         0.69592734,  0.37682483],
       [-0.28146861,  0.13270481,  0.22943848, ..., -0.64384196,
         0.6983526 ,  0.37647311]])

In [39]:
x_test_filter_LR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [40]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_LR_st.values, y_data)

In [41]:
x_data_filter.shape

(132965, 20, 11)

In [42]:
y_data_filter.shape

(132965, 6)

In [43]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_LR_st.values, y_ts_data)

In [44]:
x_ts_data_filter.shape

(45838, 20, 11)

In [45]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [46]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [47]:
x_data_filter.shape

(132965, 20, 11)

In [48]:
y_data_filter.shape

(132965, 6)

Model

In [49]:
model_with_LG_standard = LSTM_model(11)
model_with_LG_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           71680     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [50]:
model_with_LG_standard.save("modelop/LSTM/LSTM_with_LR100_standard.h5")

Evaluate

In [51]:
model = load_model("modelop/LSTM/LSTM_with_LR100_standard.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 128)           71680     
_________________________________________________________________
dropout (Dropout)            (None, 20, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [52]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.5650 - accuracy: 0.4998


[1.5650213956832886, 0.49984729290008545]

# With lowpass filter 、Polyregression and standardization (segment length = 100)

Training data with filter, Polyregression and standardization

In [53]:
def polyregression_seg(signal):
    y = signal
    X = np.arange(0, len(signal), 1)
    poly = PolynomialFeatures(degree = 2, include_bias=False)
    X_poly = poly.fit_transform(X.reshape(-1, 1))

    poly.fit(X_poly, y)
    lin2 = LinearRegression()
    lin2.fit(X_poly, y)
    yfit = lin2.predict(X_poly)
    
    return yfit

def polyregression_allsignal(signal, seg_length):
    new_signal = []
    for i in range((len(signal)//seg_length)+1):
        if (i+1)*seg_length<=len(signal):
            new_signal += polyregression_seg(signal[i*seg_length:(i+1)*seg_length]).tolist()
        else:
            new_signal += polyregression_seg(signal[i*seg_length:len(signal)]).tolist()
    
    return np.asarray(new_signal)

In [63]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 100)

In [64]:
x_train_filter_PR

array([[-172.17726401,  223.75790443,  204.65407652, ...,  429.69234858,
         260.21880471,  285.14911664],
       [-172.89022398,  222.49738985,  203.23463412, ...,  429.19631052,
         258.77218383,  283.18600527],
       [-173.58322307,  221.26808039,  201.83733044, ...,  428.70085553,
         257.34869994,  281.2494599 ],
       ...,
       [-316.81767288,  234.85455852, -262.05456808, ...,  -75.21469373,
        -147.84660063, -131.37827069],
       [-321.47456648,  235.70465045, -265.52049855, ...,  -75.79788932,
        -149.87683047, -133.37388701],
       [-326.19366005,  236.59188321, -269.01386567, ...,  -76.38474904,
        -151.9383174 , -135.39817615]])

In [65]:
x_train_filter_PR.shape

(1185498, 11)

In [66]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [67]:
x_train_filter_PR_st.values

array([[-0.95625689, -0.46669287,  2.48490557, ...,  2.05938217,
         1.36637765,  0.27642212],
       [-0.95880876, -0.47770355,  2.47026974, ...,  2.05607196,
         1.35761541,  0.27022558],
       [-0.96128918, -0.48844164,  2.45586218, ...,  2.05276563,
         1.34899332,  0.2641129 ],
       ...,
       [-1.47396266, -0.3697629 , -2.32731521, ..., -1.3100202 ,
        -1.10529041, -1.03834134],
       [-1.4906309 , -0.36233729, -2.36305233, ..., -1.31391205,
        -1.11758759, -1.04464048],
       [-1.50752177, -0.35458726, -2.39907235, ..., -1.31782835,
        -1.13007409, -1.05103012]])

In [68]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with filter, Polyregression and standardization

In [69]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 100)

In [70]:
x_test_filter_PR

array([[  67.28836454,  235.16827811, -213.47829667, ...,  112.01776968,
          41.87910261,   58.69253474],
       [  68.08449854,  235.52094773, -214.08533922, ...,  113.37859619,
          44.8634267 ,   61.74312413],
       [  68.85099469,  235.86437832, -214.67711016, ...,  114.71137395,
          47.78138547,   64.72204833],
       ...,
       [ 107.00095614,  345.63926265,  -24.33930397, ...,  -28.37433879,
          83.56787729,  260.16926227],
       [ 109.88244328,  346.08365358,  -25.27540649, ...,  -29.43236695,
          84.72582369,  263.04216621],
       [ 112.81639523,  346.53401635,  -26.22856155, ...,  -30.51527635,
          85.90328215,  265.98000891]])

In [71]:
x_test_filter_PR.shape

(236894, 11)

In [72]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [73]:
x_test_filter_PR_st.values

array([[-0.32496181, -0.9938056 , -1.93800551, ...,  0.78530828,
         0.43307123, -0.33060759],
       [-0.31944018, -0.99011718, -1.94460076, ...,  0.80095733,
         0.45809172, -0.31721038],
       [-0.31412409, -0.98652539, -1.95103009, ...,  0.81628384,
         0.48255579, -0.30412789],
       ...,
       [-0.0495327 ,  0.16156131,  0.11690656, ..., -0.82915481,
         0.78258866,  0.554214  ],
       [-0.02954797,  0.16620899,  0.10673621, ..., -0.84132178,
         0.79229684,  0.56683088],
       [-0.00919936,  0.17091914,  0.0963806 , ..., -0.85377489,
         0.80216862,  0.57973295]])

In [74]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [75]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [76]:
x_data_filter.shape

(132965, 20, 11)

In [77]:
y_data_filter.shape

(132965, 6)

In [78]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [79]:
x_ts_data_filter.shape

(45838, 20, 11)

In [80]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [81]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [82]:
x_data_filter.shape

(132965, 20, 11)

In [83]:
y_data_filter.shape

(132965, 6)

Model

In [84]:
model_with_filter_PR_standard = LSTM_model(11)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [85]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR100_standard.h5")

Evaluate

In [86]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR100_standard.h5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_3 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_5 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)              

In [87]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 11s 7ms/step - loss: 1.6065 - accuracy: 0.5099


[1.606486439704895, 0.5098826289176941]

# With lowpass filter 、Polyregression and standardization (segment length = 50)

Training data with filter, Polyregression and standardization

In [88]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 50)

In [89]:
x_train_filter_PR

array([[-207.3694826 ,  193.65720302,  213.07902993, ...,  439.59712764,
         295.87952842,  328.41857029],
       [-203.4352143 ,  196.07550237,  211.17148389, ...,  438.48870769,
         289.70100364,  321.07867343],
       [-199.66986329,  198.38353013,  209.28672692, ...,  437.38917305,
         283.75899526,  313.99066659],
       ...,
       [-330.2074663 ,  203.56457899, -207.90302292, ...,  -83.93090088,
        -153.26394529, -153.65811388],
       [-336.60394889,  198.71521548, -203.39856603, ...,  -86.02748711,
        -155.77118729, -157.94955499],
       [-343.10826886,  193.63938558, -198.60375208, ...,  -88.1931291 ,
        -158.3052294 , -162.3110021 ]])

In [90]:
x_train_filter_PR.shape

(1185498, 11)

In [91]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [92]:
x_train_filter_PR_st.values

array([[-1.07593722, -0.72220081,  2.55383813, ...,  2.10618437,
         1.56111325,  0.41041755],
       [-1.06193718, -0.70129174,  2.53430665, ...,  2.09885468,
         1.52419255,  0.38739421],
       [-1.04853824, -0.6813361 ,  2.51500851, ...,  2.09158376,
         1.4886852 ,  0.36516099],
       ...,
       [-1.51305434, -0.63653978, -1.75662236, ..., -1.35576553,
        -1.12281048, -1.10173071],
       [-1.53581613, -0.67846829, -1.71050096, ..., -1.36962969,
        -1.13779288, -1.11519184],
       [-1.55896165, -0.72235487, -1.66140657, ..., -1.3839505 ,
        -1.15293542, -1.12887255]])

In [93]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with filter, Polyregression and standardization

In [94]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 50)

In [95]:
x_test_filter_PR

array([[  98.9022333 ,  266.36599317, -204.46177503, ...,  107.75919981,
          23.64617778,   44.60135661],
       [  94.57574574,  262.59040401, -206.39183431, ...,  108.39907634,
          26.93359673,   47.52123304],
       [  90.46916893,  258.97737217, -208.24361148, ...,  109.08621453,
          30.21396177,   50.44346467],
       ...,
       [  87.88780133,  316.39941413,  -41.25326577, ...,  -20.98827899,
         111.97748229,  281.27677281],
       [  87.11118386,  311.37044077,  -44.85327602, ...,  -20.66216673,
         117.45365578,  286.80080607],
       [  86.20574331,  306.08002236,  -48.58334715, ...,  -20.29093929,
         123.13685901,  292.48368359]])

In [96]:
x_test_filter_PR.shape

(236894, 11)

In [97]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [98]:
x_test_filter_PR_st.values

array([[-0.10346207, -0.6580225 , -1.82776926, ...,  0.71874116,
         0.27357312, -0.38817354],
       [-0.13283285, -0.69694773, -1.84859861, ...,  0.7259237 ,
         0.30048222, -0.37549145],
       [-0.16071075, -0.73419704, -1.86858313, ...,  0.73363674,
         0.32733359, -0.36279913],
       ...,
       [-0.17823461, -0.14219239, -0.06641018, ..., -0.72643426,
         0.99660735,  0.63979444],
       [-0.18350676, -0.19403965, -0.10526178, ..., -0.72277368,
         1.04143247,  0.66378733],
       [-0.18965343, -0.24858234, -0.145517  , ..., -0.7186067 ,
         1.08795222,  0.68847015]])

In [99]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [100]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [101]:
x_data_filter.shape

(132965, 20, 11)

In [102]:
y_data_filter.shape

(132965, 6)

In [103]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [104]:
x_ts_data_filter.shape

(45838, 20, 11)

In [105]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [106]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [107]:
x_data_filter.shape

(132965, 20, 11)

In [108]:
y_data_filter.shape

(132965, 6)

Model

In [109]:
model_with_filter_PR_standard = LSTM_model(11)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [110]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR50_standard.h5")

Evaluate

In [111]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR50_standard.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 128)           71680     
_________________________________________________________________
dropout_6 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 20, 128)           131584    
_________________________________________________________________
dropout_7 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_8 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)              

In [112]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.7234 - accuracy: 0.4786


[1.7234293222427368, 0.47859853506088257]

# With lowpass filter 、Polyregression and standardization (segment length = 25)

Training data with filter, Polyregression and standardization

In [113]:
x_train_filter_PR = polyregression_allsignal(x_data_df.values, 25)

In [114]:
x_train_filter_PR

array([[-180.77159145,  206.7278568 ,  222.60515821, ...,  459.66327971,
         339.11245424,  382.12785358],
       [-182.81173291,  206.36714331,  219.46829425, ...,  456.13067378,
         325.95422504,  364.46502573],
       [-184.56733463,  206.10506415,  216.39671664, ...,  452.65508741,
         313.36650454,  347.72506254],
       ...,
       [-353.80567014,  165.25584067, -229.96648337, ...,  -90.26035574,
        -179.76947918, -160.66879602],
       [-373.69297366,  145.03491078, -230.87975283, ...,  -93.00046833,
        -194.89412096, -171.69742352],
       [-395.01328123,  123.28071821, -231.7721043 , ...,  -95.70808864,
        -211.18488416, -183.57632656]])

In [115]:
x_train_filter_PR.shape

(1185498, 11)

In [116]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [117]:
x_train_filter_PR_st.values

array([[-0.97530169, -0.60360124,  2.64045035, ...,  2.2247401 ,
         1.80040352,  0.57629557],
       [-0.9825172 , -0.60669142,  2.60846418, ...,  2.20152742,
         1.72259801,  0.52114016],
       [-0.98872637, -0.60893662,  2.57714371, ...,  2.17868942,
         1.64816596,  0.46886657],
       ...,
       [-1.58728392, -0.95888678, -1.97436141, ..., -1.3887959 ,
        -1.26778072, -1.11868613],
       [-1.6576208 , -1.13211695, -1.98367389, ..., -1.40680112,
        -1.35721376, -1.15312503],
       [-1.73302588, -1.31848238, -1.99277308, ..., -1.42459283,
        -1.45354216, -1.19021908]])

In [118]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with filter, Polyregression and standardization

In [119]:
x_test_filter_PR = polyregression_allsignal(x_test_df.values, 25)

In [120]:
x_test_filter_PR

array([[ 114.17482304,  275.96321018, -203.72513522, ...,  120.71047486,
          38.71786132,   39.3555655 ],
       [ 105.56163097,  268.75535528, -205.98746629, ...,  118.28202968,
          37.89098847,   41.36885898],
       [  97.53534537,  262.07441326, -208.13653896, ...,  116.14141954,
          37.4344042 ,   43.66525028],
       ...,
       [  88.30356036,  310.59315586,  -44.05697222, ...,  -37.54222551,
         102.88667737,  282.11160873],
       [  90.97775941,  302.53814344,  -49.71376518, ...,  -46.45412753,
         105.1717268 ,  289.14572361],
       [  93.96521126,  293.86275848,  -55.74674411, ...,  -56.28764509,
         107.47171817,  296.44384277]])

In [121]:
x_test_filter_PR.shape

(236894, 11)

In [122]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [123]:
x_test_filter_PR_st.values

array([[ 2.12981887e-04, -5.51931816e-01, -1.81206849e+00, ...,
         8.49497206e-01,  3.89852114e-01, -4.07781197e-01],
       [-5.70724194e-02, -6.25292872e-01, -1.83637975e+00, ...,
         8.22699404e-01,  3.83204654e-01, -3.99104322e-01],
       [-1.10454370e-01, -6.93291045e-01, -1.85947393e+00, ...,
         7.99077850e-01,  3.79534047e-01, -3.89207354e-01],
       ...,
       [-1.71853965e-01, -1.99470587e-01, -9.62562783e-02, ...,
        -8.96815486e-01,  9.05722949e-01,  6.38446742e-01],
       [-1.54068159e-01, -2.81453955e-01, -1.57044818e-01, ...,
        -9.95157995e-01,  9.24093093e-01,  6.68762311e-01],
       [-1.34198942e-01, -3.69751432e-01, -2.21875894e-01, ...,
        -1.10367050e+00,  9.42583358e-01,  7.00215683e-01]])

In [124]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [125]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [126]:
x_data_filter.shape

(132965, 20, 11)

In [127]:
y_data_filter.shape

(132965, 6)

In [128]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [129]:
x_ts_data_filter.shape

(45838, 20, 11)

In [130]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [131]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [132]:
x_data_filter.shape

(132965, 20, 11)

In [133]:
y_data_filter.shape

(132965, 6)

Model

In [134]:
model_with_filter_PR_standard = LSTM_model(11)
model_with_filter_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [135]:
model_with_filter_PR_standard.save("modelop/LSTM/LSTM_with_filter_PR25_standard.h5")

Evaluate

In [136]:
model = load_model("modelop/LSTM/LSTM_with_filter_PR25_standard.h5")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_9 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 128)              

In [137]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.4905 - accuracy: 0.4818


[1.4904829263687134, 0.4818055033683777]

# With Polyregression and standardization (segment length = 100)

Training data with Polyregression and standardization

In [138]:
x_train_filter_PR = polyregression_allsignal(x_data, 100)

In [139]:
x_train_filter_PR

array([[-178.52687245,  223.94754805,  205.76235294, ...,  428.8198544 ,
         253.46760629,  276.8306756 ],
       [-178.88900019,  222.67145903,  204.28336245, ...,  428.33815305,
         252.31768236,  275.2150691 ],
       [-179.23876418,  221.42695866,  202.82779044, ...,  427.85715323,
         251.18545393,  273.61992756],
       ...,
       [-316.00431976,  231.51246031, -262.21939421, ...,  -73.52699252,
        -147.34888109, -132.8850011 ],
       [-320.65954134,  232.19085936, -265.6848249 , ...,  -74.06395323,
        -149.38189724, -134.9921281 ],
       [-325.37748918,  232.90296846, -269.17755102, ...,  -74.60420532,
        -151.44662956, -137.13067409]])

In [140]:
x_train_filter_PR.shape

(1185498, 11)

In [141]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [142]:
x_train_filter_PR_st.values

array([[-0.97887761, -0.46501503,  2.49628836, ...,  2.05352432,
         1.3253811 ,  0.2501536 ],
       [-0.98017362, -0.47616124,  2.4810388 , ...,  2.05030983,
         1.31841652,  0.24505419],
       [-0.98142538, -0.48703154,  2.4660307 , ...,  2.04710002,
         1.31155911,  0.24001938],
       ...,
       [-1.47089195, -0.39893803, -2.32897318, ..., -1.29873535,
        -1.10218844, -1.04304814],
       [-1.48755239, -0.39301244, -2.36470451, ..., -1.30231859,
        -1.11450153, -1.04969895],
       [-1.50443733, -0.3867924 , -2.40071728, ..., -1.3059238 ,
        -1.12700671, -1.05644892]])

In [143]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with Polyregression and standardization

In [144]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 100)

In [145]:
x_test_filter_PR

array([[  66.40592895,  236.25588818, -213.46563774, ...,  113.78914968,
          43.03850903,   65.2039371 ],
       [  67.2264397 ,  236.5733171 , -214.07592912, ...,  115.06038033,
          45.96155168,   68.03868199],
       [  68.01704247,  236.88195899, -214.67086178, ...,  116.30538645,
          48.81951758,   70.80466756],
       ...,
       [ 103.63016025,  345.51137293,  -23.25581156, ...,  -25.34219965,
          84.01507714,  262.1437621 ],
       [ 106.33570603,  345.9262589 ,  -24.1197084 , ...,  -26.25491457,
          85.19004895,  265.17432856],
       [ 109.09020157,  346.34616461,  -24.99892217, ...,  -27.18969765,
          86.38479843,  268.27400616]])

In [146]:
x_test_filter_PR.shape

(236894, 11)

In [147]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [148]:
x_test_filter_PR_st.values

array([[-0.33102689, -0.982391  , -1.93783558, ...,  0.80562739,
         0.44278126, -0.30198632],
       [-0.32533711, -0.97907129, -1.94446602, ...,  0.82024524,
         0.46728741, -0.28953814],
       [-0.31985473, -0.97584348, -1.95092959, ...,  0.83456153,
         0.49124798, -0.2773919 ],
       ...,
       [-0.07289784,  0.16021634,  0.12867572, ..., -0.79424055,
         0.78631992,  0.56283306],
       [-0.0541364 ,  0.16455526,  0.11929002, ..., -0.80473584,
         0.79617062,  0.57614115],
       [-0.03503553,  0.16894669,  0.10973791, ..., -0.81548488,
         0.80618714,  0.58975272]])

In [149]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [150]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [151]:
x_data_filter.shape

(132965, 20, 11)

In [152]:
y_data_filter.shape

(132965, 6)

In [153]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [154]:
x_ts_data_filter.shape

(45838, 20, 11)

In [155]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [156]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [157]:
x_data_filter.shape

(132965, 20, 11)

In [158]:
y_data_filter.shape

(132965, 6)

Model

In [159]:
model_with_PR_standard = LSTM_model(11)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [160]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR100_standard.h5")

Evaluate

In [161]:
model = load_model("modelop/LSTM/LSTM_with_PR100_standard.h5")
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_12 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_13 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 128)              

In [162]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.7956 - accuracy: 0.4630


[1.7956410646438599, 0.463043749332428]

# With Polyregression and standardization (segment length = 50)

Training data with Polyregression and standardization

In [163]:
x_train_filter_PR = polyregression_allsignal(x_data, 50)

In [164]:
x_train_filter_PR

array([[-220.64361991,  191.84497738,  214.09230769, ...,  436.06027149,
         286.69303167,  316.02723982],
       [-215.14665805,  194.54876166,  212.13809955, ...,  435.30763321,
         281.02944316,  309.46011081],
       [-209.89095115,  197.12778142,  210.20717148, ...,  434.549228  ,
         275.59270385,  303.12504156],
       ...,
       [-326.73358686,  198.43389577, -209.19269172, ...,  -81.00019139,
        -152.01546895, -155.67886334],
       [-332.65093899,  193.06036691, -204.90270842, ...,  -82.95099327,
        -154.4699848 , -160.29449088],
       [-338.65280612,  187.43928571, -200.33290816, ...,  -84.96964286,
        -156.95153061, -164.99668367]])

In [165]:
x_train_filter_PR.shape

(1185498, 11)

In [166]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [167]:
x_train_filter_PR_st.values

array([[-1.12269422, -0.737704  ,  2.56396783, ...,  2.08261224,
         1.50571395,  0.37147893],
       [-1.1031417 , -0.71433181,  2.54396049, ...,  2.07763568,
         1.47188166,  0.35088347],
       [-1.08444732, -0.69203812,  2.52419149, ...,  2.07262099,
         1.43940449,  0.33101579],
       ...,
       [-1.50005286, -0.68074774, -1.76965812, ..., -1.33626769,
        -1.11497629, -1.10785868],
       [-1.52110069, -0.72719786, -1.72573693, ..., -1.34916668,
        -1.12963871, -1.12233395],
       [-1.54244914, -0.77578789, -1.67895094, ..., -1.36251428,
        -1.14446259, -1.1370807 ]])

In [168]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with Polyregression and standardization

In [169]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 50)

In [170]:
x_test_filter_PR

array([[  95.37674208,  267.96506787, -204.36923077, ...,  110.17701357,
          24.16434389,   59.98126697],
       [  91.42352756,  264.1364041 , -206.32262628, ...,  110.63125681,
          27.51199926,   61.62197618],
       [  87.67454012,  260.46951888, -208.19651307, ...,  111.13885216,
          30.8464355 ,   63.31039293],
       ...,
       [  79.45138668,  316.84471721,  -39.52256641, ...,  -14.9647092 ,
         113.3041305 ,  283.35617507],
       [  77.27450286,  311.82514477,  -42.85923951, ...,  -13.8775776 ,
         119.02751325,  289.22201183],
       [  74.89387352,  306.54249012,  -46.31225296, ...,  -12.70948617,
         124.9729249 ,  295.26587615]])

In [171]:
x_test_filter_PR.shape

(236894, 11)

In [172]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [173]:
x_test_filter_PR_st.values

array([[-0.12726705, -0.64141725, -1.82660636, ...,  0.74573384,
         0.2777384 , -0.32127788],
       [-0.15407712, -0.68088228, -1.84768566, ...,  0.75083168,
         0.30513312, -0.31415383],
       [-0.17950216, -0.71867973, -1.86790697, ...,  0.75652828,
         0.33241965, -0.30682263],
       ...,
       [-0.23527027, -0.1375765 , -0.04772837, ..., -0.65869499,
         1.00719187,  0.64862864],
       [-0.25003355, -0.18931715, -0.08373476, ..., -0.64649444,
         1.05402777,  0.67409844],
       [-0.2661786 , -0.2437696 , -0.1209966 , ..., -0.63338529,
         1.10268058,  0.70034124]])

In [174]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [175]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [176]:
x_data_filter.shape

(132965, 20, 11)

In [177]:
y_data_filter.shape

(132965, 6)

In [178]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [179]:
x_ts_data_filter.shape

(45838, 20, 11)

In [180]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [181]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [182]:
x_data_filter.shape

(132965, 20, 11)

In [183]:
y_data_filter.shape

(132965, 6)

Model

In [184]:
model_with_PR_standard = LSTM_model(11)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [185]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR50_standard.h5")

Evaluate

In [186]:
model = load_model("modelop/LSTM/LSTM_with_PR50_standard.h5")
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_15 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_16 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_17 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 128)              

In [187]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.5170 - accuracy: 0.4607


[1.5169981718063354, 0.46068763732910156]

# With Polyregression and standardization (segment length = 25)

Training data with Polyregression and standardization

In [188]:
x_train_filter_PR = polyregression_allsignal(x_data, 25)

In [189]:
x_train_filter_PR

array([[-209.36854701,  210.29435897,  226.73948718, ...,  455.96      ,
         322.3357265 ,  353.45230769],
       [-204.25641026,  208.68      ,  222.65230769, ...,  452.77692308,
         310.89948718,  340.46615385],
       [-199.55750279,  207.3038796 ,  218.72020067, ...,  449.64468227,
         299.98606466,  328.05892977],
       ...,
       [-348.82478826,  157.78836815, -230.27775268, ...,  -83.48170525,
        -176.09621683, -159.46674195],
       [-366.64940711,  135.74426877, -230.75533597, ...,  -83.69525692,
        -189.57351779, -167.83359684],
       [-385.67826087,  112.00434783, -231.13913043, ...,  -83.6       ,
        -204.03043478, -176.66521739]])

In [190]:
x_train_filter_PR.shape

(1185498, 11)

In [191]:
x_train_filter_PR_st = StandardScaler().fit_transform(x_train_filter_PR)
x_train_filter_PR_st = pd.DataFrame(x_train_filter_PR_st)
y=pd.DataFrame(y_data)

In [192]:
x_train_filter_PR_st.values

array([[-1.07460686, -0.57260448,  2.68164928, ...,  2.19970468,
         1.69921921,  0.48638668],
       [-1.05655723, -0.5864238 ,  2.63998775, ...,  2.17879542,
         1.63167476,  0.44586542],
       [-1.0399666 , -0.59820374,  2.59990691, ...,  2.1582201 ,
         1.56721816,  0.40715061],
       ...,
       [-1.56699081, -1.02206918, -1.97682905, ..., -1.34382525,
        -1.24460797, -1.1140973 ],
       [-1.62992493, -1.2107723 , -1.98169716, ..., -1.34522804,
        -1.3242073 , -1.14020476],
       [-1.69711089, -1.41399209, -1.98560926, ..., -1.34460231,
        -1.4095924 , -1.16776246]])

In [193]:
x_train_filter_PR_st.shape

(1185498, 11)

Testing data with Polyregression and standardization

In [194]:
x_test_filter_PR = polyregression_allsignal(x_ts_data, 25)

In [195]:
x_test_filter_PR

array([[ 106.18153846,  277.37025641, -206.44376068, ...,  121.54940171,
          39.33606838,   53.5374359 ],
       [  99.50307692,  270.45538462, -207.84051282, ...,  119.61897436,
          38.82358974,   55.52615385],
       [  93.22461538,  264.01364548, -209.21634337, ...,  117.89828317,
          38.62865106,   57.64006689],
       ...,
       [  82.3445673 ,  311.74916704,  -41.82125903, ...,  -28.99666814,
         104.66301047,  285.8004128 ],
       [  87.77042607,  304.49924812,  -46.6245614 , ...,  -34.24260652,
         108.90075188,  297.56491228],
       [  94.16090226,  296.73834586,  -51.71578947, ...,  -39.95338346,
         113.46165414,  310.37894737]])

In [196]:
x_test_filter_PR.shape

(236894, 11)

In [197]:
x_test_filter_PR_st = StandardScaler().fit_transform(x_test_filter_PR)
x_test_filter_PR_st = pd.DataFrame(x_test_filter_PR_st)
y_test=pd.DataFrame(y_ts_data)

In [198]:
x_test_filter_PR_st.values

array([[-0.05273639, -0.537134  , -1.84059905, ...,  0.85808266,
         0.39441335, -0.34624059],
       [-0.09697646, -0.60745055, -1.85560313, ...,  0.83679708,
         0.39029765, -0.33768006],
       [-0.13856682, -0.67295586, -1.87038247, ...,  0.81782412,
         0.38873211, -0.32858061],
       ...,
       [-0.21063942, -0.18753876, -0.07220453, ..., -0.801892  ,
         0.91905175,  0.65354735],
       [-0.17469695, -0.26126236, -0.12380217, ..., -0.85973558,
         0.9530849 ,  0.70418825],
       [-0.13236458, -0.3401821 , -0.17849275, ..., -0.92270465,
         0.98971335,  0.75934692]])

In [199]:
x_test_filter_PR_st.shape

(236894, 11)

Filter [0 0 0 0 0 0] from the data

In [200]:
x_data_filter, y_data_filter = filter_dataset(x_train_filter_PR_st.values, y_data)

In [201]:
x_data_filter.shape

(132965, 20, 11)

In [202]:
y_data_filter.shape

(132965, 6)

In [203]:
x_ts_data_filter, y_ts_data_filter = filter_dataset(x_test_filter_PR_st.values, y_ts_data)

In [204]:
x_ts_data_filter.shape

(45838, 20, 11)

In [205]:
y_ts_data_filter.shape

(45838, 6)

Shuffle

In [206]:
x_data_filter, y_data_filter = utils.shuffle(x_data_filter, y_data_filter)

In [207]:
x_data_filter.shape

(132965, 20, 11)

In [208]:
y_data_filter.shape

(132965, 6)

Model

In [209]:
model_with_PR_standard = LSTM_model(11)
model_with_PR_standard.fit(x_data_filter, y_data_filter, epochs=30, batch_size=64, validation_split=0.1)

C:\Users\User\anaconda3\envs\modelop_gpu\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [210]:
model_with_PR_standard.save("modelop/LSTM/LSTM_with_PR25_standard.h5")

Evaluate

In [211]:
model = load_model("modelop/LSTM/LSTM_with_PR25_standard.h5")
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 20, 128)           71680     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 128)              

In [212]:
model.evaluate(x_ts_data_filter, y_ts_data_filter)

1433/1433 [==============================] - 12s 7ms/step - loss: 1.5436 - accuracy: 0.4877


[1.5436309576034546, 0.4877176284790039]